In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
week17_link = 'http://www.footballlocks.com/nfl_odds_week_17.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week17_html = requests.get(week17_link).text

soup = BeautifulSoup(week17_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[264])

<tr>
<td>12/31 4:25 ET</td>
<td>At Baltimore</td>
<td>-8</td>
<td>Cincinnati</td>
<td>39.5</td>
<td>-$430 +$340</td>
</tr>
<tr>
<td>12/31 4:15 ET</td>
<td>At San Diego</td>
<td>-13.5</td>
<td>Arizona</td>
<td>44.5</td>
<td>-$950 +$675</td>
</tr>


In [3]:
#want to 265 because it does not include last one
table = tr[51:265]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week17_df = pd.DataFrame(string_table)

week17_df.columns = ['all_data']

week17_df.head()

,all_data
0,<tr>\n<td>12/31 4:25 ET</td>\n<td>At Baltimore...
1,<tr>\n<td>12/31 1:00 ET</td>\n<td>At Detroit</...
2,<tr>\n<td>12/31 4:25 ET</td>\n<td>Buffalo</td>...
3,<tr>\n<td>12/31 4:25 ET</td>\n<td>At Atlanta</...
4,<tr>\n<td>12/31 4:25 ET</td>\n<td>New Orleans<...


In [4]:
#want to clean the date data

dates = [x.split('td')[1] for x in week17_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week17_df['date'] = cleaned_dates

In [5]:
#pull out favorites

week17_df['cleaned_favorites'] = week17_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week17_df['cleaned_underdogs'] = week17_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week17_df['cleaned_spread'] = week17_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week17_df['cleaned_over_under'] = week17_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week17_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week17_df['money_odds'] = money_odds

#input week number for each game
week17_df['week_number'] = '17'

In [6]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2017
year_list = []
for x in week17_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week17_df['year'] = year_list

In [7]:
#create a final dataframe for week 17
week17_final = week17_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week17_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,12/31 4:25 ET,2017,17,At Baltimore,Cincinnati,-8,39.5,-$430 +$340
1,12/31 1:00 ET,2017,17,At Detroit,Green Bay,-7,45,-$350 +$290
2,12/31 4:25 ET,2017,17,Buffalo,At Miami,-2.5,41.5,-$140 +$120
3,12/31 4:25 ET,2017,17,At Atlanta,Carolina,-5,44.5,-$240 +$200
4,12/31 4:25 ET,2017,17,New Orleans,At Tampa Bay,-6,49,-$295 +$245


In [8]:
#week17_final to pickle
week17_final.to_pickle('week17_final.pkl')